In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gc
import glob
import os
import csv

import keras
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import optimizers
from keras.models import load_model
%matplotlib inline

/home/fx/env/stock/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [3]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('../Data set/FIXED_SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        temp_value = 0
        for row in file_data:
            if row[1] is '':
                continue
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = float(row[1])
                if temp_value != 0:
                    unrealize = (temp - temp_value)/ temp_value
                else:
                    unrealize = 0
                stock_data.append([unrealize])
                temp_value = temp
                
    return stock_data

In [4]:
stock_datas = [ load_stock_data(x, 2008, 2014) for x in SET50 ]

rounds = len(stock_datas[0]) - 30

In [6]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x.astype('float32')

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2] * x_train.shape[3]))

print(x_train.shape)

(1690, 36, 30) (1, 36, 30)


In [7]:
target = []
index = 0
while index < rounds:
    temp = [stock[index + 30][0] for stock in stock_datas]
    target.append(temp)
    index += 1
    
y = np.asarray(target)
y_train = y.astype('float32')

print(y_train.shape)

(1690, 36) (725, 36)


In [8]:
neurons = 512        
activation_function = 'tanh'  
loss = 'mse'                  
optimizer = 'sgd'          
dropout = 0.25                
batch_size = 100       
epochs = 10000              
# window_len = 30                     

In [9]:
def build_model(inputs, output_size, neurons, lstm_layer, activ_func=activation_function, dropout=dropout, loss=loss, optimizer=optimizer):
    model = Sequential()
    if lstm_layer == 0:
        model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2]), activation=activ_func))
        model.add(Dropout(dropout))
    if lstm_layer == 1:
        model.add(LSTM(neurons, return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2]), activation=activ_func))
        model.add(Dropout(dropout))
        model.add(LSTM(neurons, activation=activ_func))
        model.add(Dropout(dropout))
    if lstm_layer == 2:
        model.add(LSTM(neurons, return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2]), activation=activ_func))
        model.add(Dropout(dropout))
        model.add(LSTM(neurons, return_sequences=True, activation=activ_func))
        model.add(Dropout(dropout))
        model.add(LSTM(neurons, activation=activ_func))
        model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer, metrics=['acc','mae'])
    return model

In [10]:
models = [build_model(x_train, output_size=36, neurons=neurons, lstm_layer = i) for i in range(3)]

In [13]:
for index, model in enumerate(models):
    model_name = 'LSTM_Unrealize_' + str(index) + '.h5'
    model_history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose= 1)
    model.save(model_name)
# compare LSTM 1 layer vs 2 layer vs 3 layer
# train below

Epoch 1/10000
1690/1690 [==============================] - 1s 543us/step - loss: 8.1927e-04 - acc: 0.0207 - mean_absolute_error: 0.0139
Epoch 2/10000
1690/1690 [==============================] - 1s 547us/step - loss: 8.1962e-04 - acc: 0.0231 - mean_absolute_error: 0.0139
Epoch 3/10000
1690/1690 [==============================] - 1s 539us/step - loss: 8.1918e-04 - acc: 0.0237 - mean_absolute_error: 0.0138
Epoch 4/10000
1690/1690 [==============================] - 1s 544us/step - loss: 8.1966e-04 - acc: 0.0213 - mean_absolute_error: 0.0138
Epoch 5/10000
1690/1690 [==============================] - 1s 548us/step - loss: 8.1959e-04 - acc: 0.0266 - mean_absolute_error: 0.0139
Epoch 6/10000
1690/1690 [==============================] - 1s 544us/step - loss: 8.1922e-04 - acc: 0.0249 - mean_absolute_error: 0.0139
Epoch 7/10000
1690/1690 [==============================] - 1s 550us/step - loss: 8.1919e-04 - acc: 0.0243 - mean_absolute_error: 0.0139
Epoch 8/10000
1690/1690 [=======================

1690/1690 [==============================] - 1s 541us/step - loss: 8.1801e-04 - acc: 0.0314 - mean_absolute_error: 0.0139
Epoch 120/10000
1690/1690 [==============================] - 1s 536us/step - loss: 8.1780e-04 - acc: 0.0272 - mean_absolute_error: 0.0139
Epoch 121/10000
1690/1690 [==============================] - 1s 552us/step - loss: 8.1852e-04 - acc: 0.0278 - mean_absolute_error: 0.0139
Epoch 122/10000
1690/1690 [==============================] - 1s 546us/step - loss: 8.1851e-04 - acc: 0.0320 - mean_absolute_error: 0.0139
Epoch 123/10000
1690/1690 [==============================] - 1s 545us/step - loss: 8.1815e-04 - acc: 0.0337 - mean_absolute_error: 0.0139
Epoch 124/10000
1690/1690 [==============================] - 1s 548us/step - loss: 8.1793e-04 - acc: 0.0320 - mean_absolute_error: 0.0139
Epoch 125/10000
1690/1690 [==============================] - 1s 549us/step - loss: 8.1769e-04 - acc: 0.0296 - mean_absolute_error: 0.0139
Epoch 126/10000
1690/1690 [=======================

1690/1690 [==============================] - 1s 537us/step - loss: 8.1802e-04 - acc: 0.0320 - mean_absolute_error: 0.0139
Epoch 238/10000
1690/1690 [==============================] - 1s 535us/step - loss: 8.1787e-04 - acc: 0.0290 - mean_absolute_error: 0.0139
Epoch 239/10000
1690/1690 [==============================] - 1s 537us/step - loss: 8.1793e-04 - acc: 0.0302 - mean_absolute_error: 0.0139
Epoch 240/10000
1690/1690 [==============================] - 1s 537us/step - loss: 8.1767e-04 - acc: 0.0272 - mean_absolute_error: 0.0139
Epoch 241/10000
1690/1690 [==============================] - 1s 537us/step - loss: 8.1787e-04 - acc: 0.0249 - mean_absolute_error: 0.0139
Epoch 242/10000
1690/1690 [==============================] - 1s 539us/step - loss: 8.1801e-04 - acc: 0.0379 - mean_absolute_error: 0.0139
Epoch 243/10000
1690/1690 [==============================] - 1s 536us/step - loss: 8.1755e-04 - acc: 0.0225 - mean_absolute_error: 0.0139
Epoch 244/10000
1690/1690 [=======================

1690/1690 [==============================] - 1s 536us/step - loss: 8.1789e-04 - acc: 0.0302 - mean_absolute_error: 0.0139
Epoch 356/10000
1690/1690 [==============================] - 1s 536us/step - loss: 8.1802e-04 - acc: 0.0284 - mean_absolute_error: 0.0139
Epoch 357/10000
1690/1690 [==============================] - 1s 535us/step - loss: 8.1816e-04 - acc: 0.0314 - mean_absolute_error: 0.0139
Epoch 358/10000
1690/1690 [==============================] - 1s 535us/step - loss: 8.1780e-04 - acc: 0.0320 - mean_absolute_error: 0.0139
Epoch 359/10000
1690/1690 [==============================] - 1s 534us/step - loss: 8.1773e-04 - acc: 0.0254 - mean_absolute_error: 0.0139
Epoch 360/10000
1690/1690 [==============================] - 1s 535us/step - loss: 8.1733e-04 - acc: 0.0284 - mean_absolute_error: 0.0139
Epoch 361/10000
1690/1690 [==============================] - 1s 537us/step - loss: 8.1785e-04 - acc: 0.0325 - mean_absolute_error: 0.0139
Epoch 362/10000
1690/1690 [=======================

1690/1690 [==============================] - 1s 536us/step - loss: 8.1805e-04 - acc: 0.0320 - mean_absolute_error: 0.0139
Epoch 474/10000
1690/1690 [==============================] - 1s 539us/step - loss: 8.1785e-04 - acc: 0.0284 - mean_absolute_error: 0.0139
Epoch 475/10000
1690/1690 [==============================] - 1s 536us/step - loss: 8.1796e-04 - acc: 0.0284 - mean_absolute_error: 0.0139
Epoch 476/10000
1690/1690 [==============================] - 1s 539us/step - loss: 8.1758e-04 - acc: 0.0337 - mean_absolute_error: 0.0139
Epoch 477/10000
1690/1690 [==============================] - 1s 538us/step - loss: 8.1840e-04 - acc: 0.0284 - mean_absolute_error: 0.0139
Epoch 478/10000
1690/1690 [==============================] - 1s 535us/step - loss: 8.1806e-04 - acc: 0.0361 - mean_absolute_error: 0.0139
Epoch 479/10000
1690/1690 [==============================] - 1s 537us/step - loss: 8.1775e-04 - acc: 0.0284 - mean_absolute_error: 0.0139
Epoch 480/10000
1690/1690 [=======================

...

1690/1690 [==============================] - 2s 1ms/step - loss: 8.0969e-04 - acc: 0.0343 - mean_absolute_error: 0.0136
Epoch 9667/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0948e-04 - acc: 0.0337 - mean_absolute_error: 0.0136
Epoch 9668/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.1019e-04 - acc: 0.0343 - mean_absolute_error: 0.0136
Epoch 9669/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0973e-04 - acc: 0.0308 - mean_absolute_error: 0.0136
Epoch 9670/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.1032e-04 - acc: 0.0325 - mean_absolute_error: 0.0136
Epoch 9671/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0978e-04 - acc: 0.0355 - mean_absolute_error: 0.0136
Epoch 9672/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.1000e-04 - acc: 0.0290 - mean_absolute_error: 0.0136
Epoch 9673/10000
1690/1690 [==============================

1690/1690 [==============================] - 2s 1ms/step - loss: 8.0973e-04 - acc: 0.0337 - mean_absolute_error: 0.0136
Epoch 9785/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.1052e-04 - acc: 0.0367 - mean_absolute_error: 0.0136
Epoch 9786/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0981e-04 - acc: 0.0325 - mean_absolute_error: 0.0136
Epoch 9787/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0978e-04 - acc: 0.0343 - mean_absolute_error: 0.0136
Epoch 9788/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0975e-04 - acc: 0.0396 - mean_absolute_error: 0.0136
Epoch 9789/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.1012e-04 - acc: 0.0320 - mean_absolute_error: 0.0136
Epoch 9790/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0986e-04 - acc: 0.0361 - mean_absolute_error: 0.0136
Epoch 9791/10000
1690/1690 [==============================

1690/1690 [==============================] - 2s 1ms/step - loss: 8.0990e-04 - acc: 0.0331 - mean_absolute_error: 0.0136
Epoch 9903/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0942e-04 - acc: 0.0355 - mean_absolute_error: 0.0136
Epoch 9904/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0959e-04 - acc: 0.0314 - mean_absolute_error: 0.0136
Epoch 9905/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0988e-04 - acc: 0.0343 - mean_absolute_error: 0.0136
Epoch 9906/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0967e-04 - acc: 0.0361 - mean_absolute_error: 0.0136
Epoch 9907/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.0988e-04 - acc: 0.0355 - mean_absolute_error: 0.0136
Epoch 9908/10000
1690/1690 [==============================] - 2s 1ms/step - loss: 8.1021e-04 - acc: 0.0349 - mean_absolute_error: 0.0136
Epoch 9909/10000
1690/1690 [==============================

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0870e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 22/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0865e-04 - acc: 0.0284 - mean_absolute_error: 0.0135
Epoch 23/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0904e-04 - acc: 0.0373 - mean_absolute_error: 0.0135
Epoch 24/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0882e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 25/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0891e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 26/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0874e-04 - acc: 0.0391 - mean_absolute_error: 0.0135
Epoch 27/10000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0867e-04 - acc: 0.0314 - mean_absolute_error: 0.0135
Epoch 28/10000
1690/1690 [==============================] - 3s 2ms/ste

KeyboardInterrupt: 

In [9]:
model = load_model('LSTM_Unrealize_2.h5')

In [10]:
model.fit(x_train, y_train, epochs = 2000, batch_size = batch_size, verbose= 1)
# 2000, 4000, 6000

Epoch 1/2000
1690/1690 [==============================] - 4s 3ms/step - loss: 8.0795e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 2/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0790e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 3/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0790e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 4/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0812e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 5/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0781e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 6/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0817e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 7/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0805e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 8/2000
1690/1690 [==============================] - 3s 2ms/step

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0795e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 124/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0779e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 125/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0799e-04 - acc: 0.0325 - mean_absolute_error: 0.0135
Epoch 126/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0797e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 127/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0790e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 128/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0822e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 129/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0815e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 130/2000
1690/1690 [==============================] - 3s 2ms/ste

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0787e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 244/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0771e-04 - acc: 0.0367 - mean_absolute_error: 0.0135
Epoch 245/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0785e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 246/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0787e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 247/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0805e-04 - acc: 0.0325 - mean_absolute_error: 0.0135
Epoch 248/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0772e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 249/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0775e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 250/2000
1690/1690 [==============================] - 3s 2ms/ste

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0824e-04 - acc: 0.0314 - mean_absolute_error: 0.0135
Epoch 364/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0789e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 365/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0768e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 366/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0822e-04 - acc: 0.0367 - mean_absolute_error: 0.0135
Epoch 367/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0800e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 368/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0768e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 369/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0783e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 370/2000
1690/1690 [==============================] - 3s 2ms/ste

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0769e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 484/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0824e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 485/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0781e-04 - acc: 0.0325 - mean_absolute_error: 0.0135
Epoch 486/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0816e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 487/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0819e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 488/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0800e-04 - acc: 0.0308 - mean_absolute_error: 0.0135
Epoch 489/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0797e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 490/2000
1690/1690 [==============================] - 3s 2ms/ste

...

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0826e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 1444/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0804e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 1445/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0785e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1446/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0803e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 1447/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0787e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 1448/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0788e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1449/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0795e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1450/2000
1690/1690 [==============================] - 3s 

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0770e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 1564/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0756e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 1565/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0771e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 1566/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0756e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 1567/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0784e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 1568/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0795e-04 - acc: 0.0325 - mean_absolute_error: 0.0135
Epoch 1569/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0775e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 1570/2000
1690/1690 [==============================] - 3s 

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0756e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 1684/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0788e-04 - acc: 0.0367 - mean_absolute_error: 0.0135
Epoch 1685/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0796e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 1686/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0775e-04 - acc: 0.0337 - mean_absolute_error: 0.0135
Epoch 1687/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0793e-04 - acc: 0.0331 - mean_absolute_error: 0.0135
Epoch 1688/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0765e-04 - acc: 0.0367 - mean_absolute_error: 0.0135
Epoch 1689/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0788e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 1690/2000
1690/1690 [==============================] - 3s 

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0791e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 1804/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0789e-04 - acc: 0.0373 - mean_absolute_error: 0.0135
Epoch 1805/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0786e-04 - acc: 0.0343 - mean_absolute_error: 0.0135
Epoch 1806/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0802e-04 - acc: 0.0320 - mean_absolute_error: 0.0135
Epoch 1807/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0754e-04 - acc: 0.0320 - mean_absolute_error: 0.0135
Epoch 1808/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0758e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 1809/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0807e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1810/2000
1690/1690 [==============================] - 3s 

1690/1690 [==============================] - 3s 2ms/step - loss: 8.0745e-04 - acc: 0.0320 - mean_absolute_error: 0.0135
Epoch 1924/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0807e-04 - acc: 0.0302 - mean_absolute_error: 0.0135
Epoch 1925/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0766e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1926/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0807e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1927/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0779e-04 - acc: 0.0361 - mean_absolute_error: 0.0135
Epoch 1928/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0804e-04 - acc: 0.0355 - mean_absolute_error: 0.0135
Epoch 1929/2000
1690/1690 [==============================] - 3s 2ms/step - loss: 8.0763e-04 - acc: 0.0349 - mean_absolute_error: 0.0135
Epoch 1930/2000
1690/1690 [==============================] - 3s 

In [11]:
model_name = 'LSTM_Unrealize_2.h5'
model.save(model_name)